In [ ]:
'''Clear and good to go'''


#import pandas for data processing
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
def fname_ls_builder(fin_path,initial=True):
  #Find all files in data directory
  from os import listdir
  from os.path import isfile, join
  if initial:
    return [f for f in listdir(fin_path) if isfile(join(fin_path,f)) and (not ('_ds' in f or '_r' in f))]
  else:
    return [f for f in listdir(fin_path) if (isfile(join(fin_path,f)) and ('_ds' in f or '_r' in f))]

def df_builder(fin_path,fname_ls):
  #this function imports data from CSV files to one large dataframe

  #create list to hold dataframes
  df_ls=[]

  #read in each file
  for i in fname_ls:
    temp_df=pd.read_csv(fin_path+i,index_col=0)
    #print(i,'loaded\n\n')
    df_ls.append(temp_df)

  #create one large df
  df=pd.concat(df_ls)
  #print(df.shape)
  return df

def split_data(X):
  #separate y from X
  y=X['label']
  X.drop(['label'],axis=1,inplace=True)
  #split into train and dev sets
  #print(X,'\n\n',y)
  return X,y


#set filepaths for import/export
pkin_path=r'/content/drive/My Drive/ML Spectroscopy/Data/Preprocessed/Peaks Only/Labeled/'

pk_fname_ls=fname_ls_builder(pkin_path)

pk_df=df_builder(pkin_path,pk_fname_ls)

#drop relative intensities
pk_df.drop(columns=[i for i in pk_df.columns.values if 'val' in i],inplace=True)

pk_df,y=split_data(pk_df)


In [ ]:
#split into train and dev sets
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(pk_df, y, test_size = 0.20, random_state = 0)
#print(X_train,'\n\n',X_dev,'\n\n',y_train,'\n\n',y_dev)



In [ ]:
#scale data((X-mean)/std_dev)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_dev_sc = sc.fit_transform(X_dev)
#print(X_train_sc,'\n\n',X_dev_sc)

In [ ]:
from sklearn.decomposition import PCA
#run PCA
X_train_p=PCA(n_components=3).fit_transform(X_train_sc)
X_dev_p=PCA(n_components=3).fit_transform(X_dev_sc)
#print(X_train_p,'\n\n',X_dev_p)

ValueError: ignored

In [ ]:
#create model
from sklearn.svm import LinearSVC
model=LinearSVC(penalty='l1',dual=False,max_iter=5000).fit(X_train_sc,y_train)
modelp=LinearSVC(penalty='l1',dual=False,max_iter=5000).fit(X_train_p,y_train)

In [ ]:
#check model
print(model.score(X_dev_sc,y_dev))
print(modelp.score(X_dev_p,y_dev))

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_dev,model.predict(X_dev_sc))
pcm=confusion_matrix(y_dev,modelp.predict(X_dev_p))
print(cm,'\n\n\n',pcm)

In [ ]:
testin_path=r'/content/drive/My Drive/ML Spectroscopy/Data/Preprocessed/Peaks Only/Unlabeled/'
test_ls=fname_ls_builder(testin_path)
test_df=df_builder(testin_path,test_ls)
test_df.drop(columns=[i for i in test_df.columns.values if 'val' in i],inplace=True)
X_test=sc.fit_transform(test_df)
X_test_p=PCA(n_components=3).fit_transform(X_test)

saved_data=r'/content/drive/My Drive/ML Spectroscopy/Programs/Data Processing/Saved Lists/'

pd.DataFrame(data=model.decision_function(X_test),index=test_df.index.values,columns=sorted(y.unique())).to_csv(saved_data+r'linear_svc_non-synth.csv')
pd.DataFrame(data=modelp.decision_function(X_test_p),index=test_df.index.values,columns=sorted(y.unique())).to_csv(saved_data+r'linear_svc_non-synth_pca.csv')
